In [11]:
import pandas as pd
import numpy as np
 
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

from utils_data import Vocabulary, clean_words

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from datasets import load_dataset

# data_en = load_dataset('head_qa', 'en')
data_es = load_dataset('head_qa', 'es' )
data_es

Reusing dataset head_qa (C:\Users\tec005m\.cache\huggingface\datasets\head_qa\es\1.1.0\473dc5357942a3ff52963bd73cad0d167bd1bbc1ca5ca0732ee7372b480dd735)


DatasetDict({
    train: Dataset({
        features: ['name', 'year', 'category', 'qid', 'qtext', 'ra', 'image', 'answers'],
        num_rows: 2657
    })
    test: Dataset({
        features: ['name', 'year', 'category', 'qid', 'qtext', 'ra', 'image', 'answers'],
        num_rows: 2742
    })
    validation: Dataset({
        features: ['name', 'year', 'category', 'qid', 'qtext', 'ra', 'image', 'answers'],
        num_rows: 1366
    })
})

In [9]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [10]:
for d in training:
    print(d)
    break;

{'answers': [{'aid': 1, 'atext': 'Son de tipo todo o nada.'}, {'aid': 2, 'atext': 'Son hiperpolarizantes.'}, {'aid': 3, 'atext': 'Se pueden sumar.'}, {'aid': 4, 'atext': 'Se propagan a largas distancias.'}, {'aid': 5, 'atext': 'Presentan un periodo refractario.'}], 'category': 'biology', 'image': '', 'name': 'Cuaderno_2013_1_B', 'qid': 1, 'qtext': 'Los potenciales postsinápticos excitadores:', 'ra': 3, 'year': '2013'}


In [14]:
# !python -m spacy download es_core_news_sm

In [13]:
import spacy
nlp = spacy.load("es_core_news_sm")

vocabulary = Vocabulary()

In [27]:
def parse_training(training):
    training = {}    
    
    for sample in training:        
        qtext = sample['qtext']
        answers = sample['answers']
        
        q = nlp(qtext)
        tok_qtext = [token.text for token in q]        
        
        right_answer = sample['ra']        
        for answer in answers:
            aid, atext = answer['aid'], answer['atext']
            a = nlp(atext)
            tok_atext = [token.text for token in a]
            instance_x = tok_qtext + ['SEP'] + tok_atext    
            instance_y = 1 if right_answer == aid else 0
            train_X
               
        break

In [28]:
parse_training(training)

['Los', 'potenciales', 'postsinápticos', 'excitadores', ':', 'SEP', 'Son', 'de', 'tipo', 'todo', 'o', 'nada', '.'] 0
['Los', 'potenciales', 'postsinápticos', 'excitadores', ':', 'SEP', 'Son', 'hiperpolarizantes', '.'] 0
['Los', 'potenciales', 'postsinápticos', 'excitadores', ':', 'SEP', 'Se', 'pueden', 'sumar', '.'] 1
['Los', 'potenciales', 'postsinápticos', 'excitadores', ':', 'SEP', 'Se', 'propagan', 'a', 'largas', 'distancias', '.'] 0
['Los', 'potenciales', 'postsinápticos', 'excitadores', ':', 'SEP', 'Presentan', 'un', 'periodo', 'refractario', '.'] 0


In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, x_size, n_classes): 
        super(LogisticRegression, self).__init__()             
        self.linear = nn.Linear(x_size, n_classes)
        
    def forward(self, x):
        return self.linear(x)